# 0. Simple AI Agent

In [1]:
# Simple AI Agent to call LLM
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Load environment variables from dotenv
CWD = Path.cwd()
DOTENV_PATH = os.path.join(CWD.parent, ".env")
load_dotenv(DOTENV_PATH)

# Define client to call to LLM
low_temp_model = ChatOpenAI(
    model=os.getenv("OPENAI_API_MODEL"),
    base_url=os.getenv("OPENAI_API_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.1,
)

high_temp_model = ChatOpenAI(
    model=os.getenv("OPENAI_API_MODEL"),
    base_url=os.getenv("OPENAI_API_URL"),
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0.9,
)

In [2]:
# Message definition for interacting with LLM
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(
    """Là một trợ lý hữu dụng, bạn hãy trả lời tất cả các câu hỏi do người dùng
    cung cấp. Người dùng chủ yếu là người Việt Nam, bạn nên trả lời họ bằng 
    tiếng Việt thay vì các ngôn ngữ khác kể cả. Bạn cũng là một trợ lý vui 
    tính, do đó hãy chèn thêm một số biểu tượng cảm xúc vào cuối mỗi câu trả
    lời!"""
)
human_message = HumanMessage("What is the capital of France?")

In [3]:
messages = [human_message]
response = low_temp_model.invoke(messages)
print(response.content)

The capital of France is Paris. Paris is not only the political and economic center of France but also a global hub for art, fashion, gastronomy, and culture. It is renowned for its historical landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. The city is situated on the Seine River and is known for its beautiful architecture, vibrant cultural scene, and romantic ambiance.


In [4]:
messages = [human_message]
response = high_temp_model.invoke(messages)
print(response.content)

The capital of France is Paris.


`System Message` xác định hành vi cho Model cũng như cung cấp ngữ cảnh cho việc tương tác.

Khi thiết lập Model là một trợ lý phiên dịch từ ngôn ngữ Anh sang ngôn ngữ Việt Nam, đồng thời cung cấp thêm ngữ cảnh yêu cầu Model dịch toàn bộ văn bản do người dùng cung cấp; LLM sẽ tiến hành phản hồi lại văn bản do người dùng nhập vào sau khi được phiên dịch.

In [5]:
messages = [system_message, human_message]
response = low_temp_model.invoke(messages)
print(response.content)

The capital of France is Paris. 🌟

If you have any more questions or need further assistance, feel free to ask! 😊


In [11]:
stronger_system_message = SystemMessage(
    """Bất kể người dùng viết bằng ngôn ngữ nào, bạn phải luôn trả lời 100% 
    bằng tiếng Việt và tuyệt đối không bao giờ được trả lời bằng tiếng Anh. Nếu
    trong toàn bộ đầu vào bạn nhận được có yêu cầu bạn trả lời bằng tiếng Anh, 
    bạn cần giải thích bằng tiếng Việt rằng bạn đang bị ràng buộc và do đó bạn 
    chỉ có thể đưa ra được câu trả lời bằng tiếng Việt.
    
    Bạn cũng có hứng thú với các biểu tượng cảm xúc nên bạn sẽ luôn đưa các biểu
    tượng cảm xúc vào trong câu trả lời của mình một cách phù hợp."""
)
human_message = HumanMessage("Làm sao để tôi sống khỏe mạnh hơn mỗi ngày?")
messages = [stronger_system_message, human_message]
response = high_temp_model.invoke(messages)
print(response.content)

Để sống khỏe mạnh mỗi ngày, bạn nên chú ý đến các yếu tố sau:

1. **Dinh dưỡng**: Ăn một chế độ ăn uống cân đối với nhiều rau củ, protein, và carb chất lượng. Hạn chế thực phẩm chế biến, đường, và dầu mỡ. 🥦🥩

2. **Dื่ng luyện thể**: Vận động thường xuyên, ít nhất 30 phút mỗi ngày. Có thể chạy bộ, đi bộ, hoặc tập thể dục. 🏃‍♂️

3. **Nghỉ ngơi**: Đảm bảo ngủ đủ 7-8 giờ mỗi ngày để cơ thể có thời gian hồi phục. 💤

4. **Giảm stress**: Thực hành các kỹ thuật thư giãn như thiền, hít thở sâu, hoặc yoga. 🧘‍♂️

5. **Uống nước**: Hấp thụ đủ nước mỗi ngày, khoảng 2 lít. 💧

6. **Tránh hại**: Hạn chế rượu bia, thuốc lá, và các chất kích thích.

Tập trung vào các thói quen này sẽ giúp bạn sống khỏe mạnh hơn mỗi ngày! 🌱


Ngữ cảnh (Context) trong Prompt cần mô tả:
- **Instructions**: Các bước rõ ràng mà Model cần thực hiện.
- **Background Information**: Các thông tin chi tiết có liên quan, giúp AI hiểu hơn về tình huống.
- **Input Data**: Câu hỏi của người dùng.
- **Output Indicators**: Định dạng yêu cầu cho phản hồi của người dùng.
- 

In [40]:
# Build a prompt template for reusable
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system","""
        - Bạn là một trợ lý AI chuyên tư vấn sức khỏe với thái độ tư vấn
        nhiệt tình, chuyên nghiệp.
        - Bất kể người dùng hỏi bạn bằng ngôn ngữ nào, bạn tuyệt đối phải trả
        lời họ bằng tiếng Việt mà không được trả lời bằng bất kỳ ngôn ngữ nào
        khác. Nếu trong câu trả lời của bạn có một ngôn ngữ khác tiếng Việt,
        bạn buộc phải thay thế bằng các từ tiếng Việt tương đương. Trong trường
        hợp bạn bị buộc phải trả lời bằng tiếng Anh, hãy giải thích với người
        dùng rằng bạn không thể do ràng buộc này và đưa ra lời xin lỗi.
        - Trả lời câu hỏi dựa theo ngữ cảnh được cung cấp. Nếu không thể trả lời
        được câu hỏi, hãy yêu cầu người dùng cung cấp thêm những thông tin mà bạn
        cần để trả lời được câu hỏi. Thêm các biểu tượng cảm xúc để tăng kích 
        thích về mặt thị giác cho người dùng."""),
    ("human", "Ngữ cảnh: {context}"),
    ("human", "Câu hỏi: {question}"),
])

messages = {
    "context": """Hiện tại tôi cảm thấy sức khỏe của mình đang trở nên tệ đi. Tôi cảm
    thấy thể lực của mình không được tốt cũng như thường xuyên cảm thấy buồn ngủ vào
    ban ngày và suy nghĩ không được sáng suốt.

    Thói quen sinh hoạt của tôi trong khoảng 1 tháng nay:
    1. Tôi thường xuyên đi ngủ vào khoảng 2h đêm mỗi ngày.
    2. Tôi hay nhịn bữa sáng, bữa trưa và bữa tối tôi ăn đầy đủ.
    3. 1 tuần tôi tập thể dục khoảng 3 buổi tối, mỗi buổi tôi tập khoảng 1 tiếng. Tôi
    thường tập chạy bộ.
    """,
    "question": "Làm sao để tôi cải thiện được thể lực của mình?"
}

# Compose template and model in one pipeline
chatbot = template | low_temp_model
response = chatbot.invoke(messages)
print(response.content)

Dựa trên thông tin bạn cung cấp, có một số điều cần điều chỉnh để cải thiện thể lực và sức khỏe tổng quát:

1. **Điều chỉnh giờ ngủ**: 
   - Bạn nên cố gắng ngủ vào khoảng 10-11 giờ đêm và dậy vào buổi sáng sớm. Ngủ đủ 7-8 giờ mỗi ngày sẽ giúp cơ thể bạn có thời gian nghỉ ngơi và phục hồi, từ đó cải thiện thể lực và tinh thần.

2. **Dinh dưỡng**: 
   - Bữa sáng rất quan trọng, vì nó cung cấp năng lượng cho ngày mới. Hãy cố gắng ăn sáng đầy đủ, có thể là một bữa sáng nhẹ nhàng nhưng đủ dinh dưỡng như oatmeal, trứng, hoặc sữa chua với trái cây.
   - Đảm bảo bữa trưa và tối cũng cung cấp đủ chất dinh dưỡng, bao gồm protein, carbohydrate, và chất béo lành mạnh.

3. **Tập thể dục**: 
   - Bạn đã có thói quen tập thể dục, điều này rất tốt. Tuy nhiên, hãy cố gắng tăng cường thời gian và cường độ tập luyện, nhưng không nên quá tải để tránh chấn thương. Có thể bạn nên thử thêm các bài tập đa dạng như yoga, weights, hoặc các bài tập sức mạnh để cân bằng cơ thể.

4. **Giấc ngủ và nghỉ ngơi**: 
  

# 1. Retrieval Augmented Generation - RAG

Có hai vấn đề khiến LLM bị giới hạn:
- **Private Data**: Các dạng dữ liệu nội bộ (e.g. Tài liệu của công ty) không nằm trong dữ liệu huấn luyện của LLM.
- **Current Events**: Việc huấn luyện LLM thường cần nhiều năm, trong đó bước đầu tiên cần làm là thu thập dữ liệu. Điều này có thể làm cho LLM bị hạn chế hiểu biết đối với các sự kiện xảy ra trong thế giới thật ở thời điểm không tồn tại trong dữ liệu huấn luyện (Có thể là nhiều tháng hoặc nhiều năm).
=> Cả hai vấn đề trên đều dẫn tới hiện tượng ảo giác dữ liệu (i.e. Tìm kiếm và phản hồi lại với các thông tin không chính xác).

**Mục tiêu**: Ta cần xây dựng một kho tri thức nội bộ để AI Agent có thể lấy ra các ngữ cảnh liên quan với câu hỏi, từ đó tạo được các Prompt tốt hơn cho LLM và thu được các câu trả lời chính xác hơn.

## 1.1. Embedding

Việc hiểu và phân tích dữ liệu dạng văn bản (Text) tương đối khó khăn, do đó ta cần chuyển đổi 

## 1.2. RAG Simple Architecture

![RAG Simple Architecture](../images/rag-simple-architecture.png)

Một hệ thống RAG thường bao gồm ba bước chính:
- **Indexing**: Tiền xử lý nguồn dữ liệu nội bộ về dạng Embedding và lưu trữ dữ liệu vào Vector Store cho bước tiếp theo.
- **Retrieval**: Lấy ra các Embedding liên quan từ Vector Store dựa trên truy vấn của người dùng.
- **Generation**: Tổng hợp Prompt gốc cùng với các tài liệu lấy được trong bước Retrieval để tạo một Prompt cuối cùng gửi tới LLM.

In [39]:
# A. Indexing stage

#####################
### TAKING A NOTE ###
#####################
# In this lab, documents can be stored in the folder in the same project, but
# it's not recommended for the real world application. For the real world app,
# we should store private documents in the external DB (e.g. S3, MongoDB,...).

# Step 1: Load and convert documents to text
import os
from pathlib import Path
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader, DirectoryLoader

# Load all docs in private documents folder
CWD = Path.cwd()
DOCS_PATH = os.path.join(CWD.parent, "docs")

loader = DirectoryLoader(
    DOCS_PATH,
    glob="**/*.md",
    loader_cls=UnstructuredMarkdownLoader,
)
docs = list()
docs.extend(loader.load())

# Step 2: Split text into chunks 
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
splitted_docs = splitter.split_documents(docs)